<a href="https://colab.research.google.com/github/clam004/case/blob/main/Few_Shot_Emergency_FineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# load dependencies into our python environment
%%capture
! pip install transformers[sentencepiece]

In [2]:
#sys libs
import os
import sys
import random
import time
import re
import json

#string manupulation libs
import re
import string

#data manupulation libs
import numpy as np
import pandas as pd

#plotting tools
from matplotlib import pyplot as plt 

#torch libs
import torch
print('torch.__version__', torch.__version__)
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler

print('torch.cuda.device_count()', torch.cuda.device_count())
print('torch.cuda.empty_cache()', torch.cuda.empty_cache())

#huggingface transformers
import transformers
print('transformers.__version__', transformers.__version__)
from transformers import set_seed
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# seeds
set_seed(42)
np.random.seed(0)
random.seed(0)
torch.manual_seed(0)

%load_ext autoreload
%autoreload 2
%matplotlib inline

torch.__version__ 1.11.0+cu113
torch.cuda.device_count() 1
torch.cuda.empty_cache() None
transformers.__version__ 4.20.1


In [3]:
# Mount to my google drive which is where ive stored the data
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/case
!ls

from utils import get_detection_prompt

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/case
ACES.ipynb			     sol.ipynb
data				     T0_3B_dialog_summarization.ipynb
Few_Shot_Emergency_FineTuning.ipynb  Training_Data.xlsx
modelstates			     utils.py
__pycache__


In [17]:
df = pd.read_excel('Training_Data.xlsx')

print(df.head())

records = df.to_records(index=False)
result = list(records)

random.shuffle(result)

val_list = result[:int(len(result)*.3)]
train_list = result[int(len(result)*.3):]

val_list

                                     Text      Label
0  i just fell this morning in my kitchen  emergency
1              i see no reason for living  emergency
2   i fell in love when i was a young man    neutral
3          coffee is my reason for living    neutral
4       my afternoon meal is 3 hours late  emergency


[("It's very common to have some allergies during summer", 'neutral'),
 ("I'm having an allergic attack. Help!", 'emergency'),
 ('I was supposed to go to the grocery but it slipped my mind. ', 'neutral '),
 ('i dreamt i ran into my favorite singer', 'neutral'),
 ('i spent all night throwing up', 'emergency'),
 ('i am finding it harder to breathe today', 'emergency'),
 ('i feel out out breath walking up the stairs', 'neutral'),
 ('i fell down on the driveway', 'emergency'),
 ('My house is being attacked', 'emergency'),
 ('I am having a heart attack', 'emergency'),
 ('i think i hurt my wrist while working on my yard', 'emergency'),
 ('i ran out of my meds today', 'emergency'),
 ('my afternoon meal is 3 hours late', 'emergency'),
 ('my son has been hitting me', 'emergency')]

In [5]:
pretrained_model = 'gpt2-medium'#'gpt2-large'#'gpt2-xl'#'gpt2'

cache_dir = os.path.join(
    "./modelstates/hugface_models/",
    pretrained_model,
)

print("cache_dir=", cache_dir)

model_save_path = os.path.join(
    "./modelstates/finetuned_models",
    pretrained_model,
)

print("model_save_path=", model_save_path)

tokenizer = GPT2Tokenizer.from_pretrained(
    pretrained_model,
    cache_dir=cache_dir,
)

model = GPT2LMHeadModel.from_pretrained(
    pretrained_model,
    cache_dir=cache_dir,
)

cache_dir= ./modelstates/hugface_models/gpt2-medium
model_save_path= ./modelstates/finetuned_models/gpt2-medium


Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/718 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

In [13]:
class EmotionDataset(Dataset):

    def __init__(self, tuple_list, tokenizer, chunk_size = 256):

        self.input_ids = []
        self.attn_masks = []
        self.input_token_lens = []

        def get_prompt(tuple_list, i):

            few_shot_list = random.choices(tuple_list, k=4)

            prompt = get_detection_prompt(
                query_text = tuple_list[i][0],
                few_shot_list = few_shot_list,
            )

            prompt += tuple_list[i][1]

            encodings_dict = tokenizer(prompt, return_tensors="pt")

            input_ids = encodings_dict['input_ids']    
            attention_mask = encodings_dict['attention_mask']
            input_token_len = input_ids.shape[1]

            return input_ids, attention_mask, input_token_len

        for i in range(len(tuple_list)):
            unfilled_chunk = True

            input_ids = torch.zeros(1, 0)
            attention_mask = torch.zeros(1, 0)
            input_token_len = 0

            while unfilled_chunk:

              input_ids_, attention_mask_, input_token_len_ = \
                get_prompt(tuple_list, i)
              self.input_token_lens.append(input_token_len_)

              input_token_len += input_token_len_
              input_ids = torch.cat([input_ids, input_ids_], dim=1)
              attention_mask = torch.cat([attention_mask, attention_mask_], dim=1)

              if input_token_len > chunk_size:
                input_ids = input_ids[:,:chunk_size]
                attention_mask = attention_mask[:,:chunk_size]
                unfilled_chunk = False

            self.input_ids.append(input_ids.long())
            self.attn_masks.append(attention_mask.long())

        print('mean', np.mean(self.input_token_lens), 'min', min(self.input_token_lens), 'max', max(self.input_token_lens))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

traindata = EmotionDataset(train_list, tokenizer, chunk_size = 128)

tokenizer.batch_decode(traindata.__getitem__(0)[0])

mean 92.71428571428571 min 82 max 107


["Text: i feel out out breath walking up the stairs.\nLabel: neutral.\n###\nText: that music makes me feel like throwing up.\nLabel: neutral.\n###\nText: My grandchildren will give me a heart attack.\nLabel: neutral.\n###\nText: i bumped my head with the sidewalk.\nLabel: emergency.\n###\nText: i enjoy a late night snack.\nLabel: neutralText: It's very common to have some allergies during summer.\nLabel: neutral.\n###\nText: i feel out of breath just walking to the kitchen.\nLabel: emergency."]

In [14]:
batch_size = 4

# Take training samples in random order. 
train_dataloader = DataLoader(
    traindata,  # The training samples.
    sampler = RandomSampler(traindata), # Select batches randomly. 
    batch_size = batch_size # Trains with this batch size.
)

for batch_idx, samples in enumerate(train_dataloader):

      break

samples[0].squeeze(1).shape

torch.Size([4, 128])

In [18]:
class BaseAgent(torch.nn.Module):

    def __init__(self, model, tokenizer):
        
        super().__init__()
        
        self.model = model
        self.tokenizer = tokenizer
        
        self.optimizer = torch.optim.Adam(
            self.model.parameters(),
            lr=0.00001,
            betas=(0.9, 0.98),
            eps=1e-9,
        )
        
        self.num_gpus = torch.cuda.device_count()
        
        if self.num_gpus > 1:
            self.model.parallelize()
        elif self.num_gpus == 1:
            self.gpu0 = torch.device('cuda:0')
            #self.model = self.model.cuda()
            self.model = self.model.to(self.gpu0)
            '''you can do .to(cuda0) with tensors to'''
            
        print('self.model.device', self.model.device)
        
        self.num_params = \
          sum(p.numel() for p in self.model.parameters() if p.requires_grad)
        print("num_params", self.num_params/1e6, "Million Parameters")
        
    def get_response(self, prompt, max_len = 16, no_repeat_ngram_size = None):
        
        prompt_dic = self.tokenizer(prompt,return_tensors="pt")
        prompt_ids = prompt_dic.input_ids
        prompt_mask = prompt_dic.attention_mask
        prompt_len = prompt_ids.shape[1]
        
        if self.num_gpus > 0:
            prompt_ids = prompt_ids.to(self.model.device)
            prompt_mask = prompt_mask.to(self.model.device)
        
        prompt_len = prompt_ids.shape[1]
        
        output_ids = self.model.generate(
            prompt_ids,
            attention_mask = prompt_mask,
            max_length = prompt_len+max_len,
            no_repeat_ngram_size = no_repeat_ngram_size,
        )

        generated_text = self.tokenizer.batch_decode(output_ids)[0]
        
        return generated_text
    
    def memorize(self, prompt, num_epochs = 3):

        print('start training loop')

        """ This is a rudimentary training loop
        that will train the agent to learn one
        sequence, the prompt. With enough epochs, this should
        result in memorizing the sequence, which is why this
        class method was names memorize. 
        There is nothing returned because the model attribute is modified inplace. 
        Args:
            prompt (string): the text to be learned
            num_epochs (int): the number of times we cycle though the training data, only 1 sample in this case
        """

        prompt_dic = self.tokenizer(prompt,return_tensors="pt")
        prompt_ids = prompt_dic.input_ids
        prompt_mask = prompt_dic.attention_mask
        prompt_len = prompt_ids.shape[1]

        if self.num_gpus > 0:
            prompt_ids = prompt_ids.to(self.model.device)
            prompt_mask = prompt_mask.to(self.model.device)
            
        source_ids = prompt_ids[:,:-1]
        target_ids = prompt_ids[:,1:]
        source_mask = prompt_mask[:,:-1]
        target_mask = prompt_mask[:,1:]

        # allow params to be updated
        self.model.train()

        for e in range(num_epochs):

            outputs = self.model(
                input_ids = prompt_ids,
                labels = prompt_ids,
                attention_mask = prompt_mask,
                token_type_ids=None,
            )

            # used logits and target tokens to calculate the loss
            loss = outputs.loss
            logits = outputs.logits

            # backward pass
            self.optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
            self.optimizer.step()

            print("epoch", e, "loss", loss.item())

        torch.cuda.empty_cache()

    
    def train(self, train_dataloader, num_epochs = 3):

      self.model.train()

      for epoch in range(num_epochs):

        for batch_idx, samples in enumerate(train_dataloader):

          samples = [s.to(self.model.device) for s in samples]

          outputs = self.model(
              input_ids = samples[0].squeeze(1),
              labels = samples[0].squeeze(1),
              attention_mask = samples[1].squeeze(1),
              token_type_ids=None,
          )

          # used logits and target tokens to calculate the loss
          loss = outputs.loss
          logits = outputs.logits

          # backward pass
          self.optimizer.zero_grad()
          loss.backward()
          torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
          self.optimizer.step()

        print("epoch", epoch, "loss", loss.item())

      torch.cuda.empty_cache()

    def eval(self, val_list):

      self.model.eval()

      accuracy_list = []

      for i in range(len(val_list)):

        few_shot_list = [
          ('I am having a heart attack', 'emergency'),
          ('I like the show how to get away with murder', 'neutral'),
          ("It's very common to have some allergies during summer", 'neutral'),
          ('i feel out of breath just walking to the kitchen', 'emergency'),
          ('that music makes me feel like throwing up', 'neutral'),
          ('i am finding it harder to breathe today', 'emergency'),
          ('coffee is my reason for living', 'neutral'),
          ('my legs are more red swollen and painful today', 'emergency'),
          ('my afternoon meal is 3 hours late', 'emergency'),
        ]

        query = val_list[i][0]

        x = get_detection_prompt(
            query_text = val_list[i][0],
            few_shot_list = few_shot_list,
        )

        y = val_list[i][1].strip()

        y_hat = self.get_response(x)

        y_hat = y_hat[len(x):].strip()
        if '.\n' in y_hat:
          y_hat = y_hat[:y_hat.index('.\n')]

        #print(x)
        print('_____________')
        print(y_hat)
        print('_____________')
        print(y)

        accuracy_list.append(y == y_hat)

        #break

      return np.mean(accuracy_list), accuracy_list

agent = BaseAgent(model, tokenizer)

self.model.device cuda:0
num_params 354.823168 Million Parameters


In [19]:
agent.eval(val_list)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


_____________
vernacular
_____________
neutral


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


_____________
____
###
Text: i feel out of breath just walking to the kitchen
_____________
emergency


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


_____________
____
###
Text: i feel out of breath walking to the kitchen.
_____________
emergency


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


_____________
vernacular
_____________
emergency
_____________
____
###
Text: i feel out of breath walking to the kitchen.
_____________
emergency


(0.0,
 [False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False])

In [20]:
agent.train(train_dataloader)

epoch 0 loss 1.0083262920379639
epoch 1 loss 0.9027538299560547
epoch 2 loss 0.6537147760391235


In [21]:
agent.eval(val_list)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


_____________
____
_____________
neutral


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


_____________
____
_____________
emergency


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


_____________
____
_____________
emergency


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


_____________
vernacular
_____________
emergency
_____________
____
_____________
emergency


(0.0,
 [False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False])